# Accessing the Youtube API
This Notebook explores convenience functions for accessing the Youtube API.
Writen by Leon Yin and Megan Brown

In [1]:
import os
import sys
import json
import datetime
import pandas as pd

# this is to import youtube_api from the py directory
sys.path.append(os.path.abspath('../youtube-data-api')) 
from youtube_api import YoutubeDataApi, get_upload_playlist_id, P
import youtube_api_utils 

from runtimestamp.runtimestamp import runtimestamp
runtimestamp()

Updated 2018-07-01 18:08:33.886037
By None
Using Python 3.6.5
On Windows-10-10.0.17134-SP0


In [7]:
key = os.environ.get('YT_API')

We can go from Username to `channel_id`. `channel_id` is required to get uploaded videos, user metadata, and relationships like subscriptions and featured channels.

In [6]:
yt = YoutubeDataApi(key)

In [4]:
yt.get_channel_id_from_user('munchies')

'UCaLfMkkHhSA_LaCta0BzyhQ'

In [5]:
channel_id = 'UCaLfMkkHhSA_LaCta0BzyhQ'

We can collect channel-level metrics and metadata:

In [6]:
channel_meta = yt.get_channel_metadata(channel_id)
channel_meta

OrderedDict([('id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
             ('title', 'Munchies'),
             ('publish_date', datetime.datetime(2014, 3, 24, 21, 21, 29)),
             ('keywords',
              'food vice "how to" cooking recipe "fresh off the boat" munchies eating "epic mealtime" tutorial "cooking show"'),
             ('description',
              'MUNCHIES is a website and digital video channel from VICE dedicated to food and its global purpose. Launched in 2014, MUNCHIES offers groundbreaking content from a youth driven perspective. In today\'s modern world, the formerly tangible pleasures of music, film, and emerging media are just one click away. Food and the events that manifest around it are one of the everlasting experiences that cannot be replicated by arcs and zeroes. MUNCHIES chronicles the wide spectrum of the global culinary experience and the diverse voices that are pulling us forward: chefs and home cooks, makers and consumers, the politics and policies of food, "

Note for some API calls that require a "playlist ID", you need to use the playlist id (from uploads or likes) rather than the `channel_id`.

In [7]:
playlist_id = channel_meta['playlist_id_uploads']
playlist_id

'UUaLfMkkHhSA_LaCta0BzyhQ'

In [8]:
channel_id == playlist_id

False

For user uploads the channel ID's first two letters are replaced with "UU" (User Upload) and "LL" (Likes), <br>these relationships are captured in two helper-functions.<br> `yt.get_upload_playlist_id()` and `yt.get_liked_playlist_id()`

In [9]:
get_upload_playlist_id(channel_id)

'UUaLfMkkHhSA_LaCta0BzyhQ'

We can use the following function to get all the video IDs from any playlist id.<br>
`cutoff_date` can be used to filter out videos after a certain date and `stop_after_n_iterations` can be used for testing to return the first N * 50 video ids.

In [10]:
video_ids = yt.get_video_urls_from_playlist_id(playlist_id, key,
                                               cutoff_date=datetime.datetime(2017,1,1))

>> 50 Videos parsed. Next Token = CDIQAA
>> 100 Videos parsed. Next Token = CGQQAA
>> 129 Videos parsed. Next Token = CJYBEAA


In [11]:
df = pd.DataFrame(video_ids)
df.head()

publish_date     video_id                channel_id  \
0 2018-05-30 15:01:36  IEDp9GQPk2I  UCaLfMkkHhSA_LaCta0BzyhQ   
1 2018-05-26 14:00:04  zi1xZhE9HFQ  UCaLfMkkHhSA_LaCta0BzyhQ   
2 2018-05-19 15:00:03  iA9CFJeftJU  UCaLfMkkHhSA_LaCta0BzyhQ   
3 2018-05-23 11:00:06  eT9QeDqZbWc  UCaLfMkkHhSA_LaCta0BzyhQ   
4 2018-05-22 11:00:10  ktZkf8Hfjd4  UCaLfMkkHhSA_LaCta0BzyhQ   

             collection_date  
0 2018-05-31 22:31:39.735375  
1 2018-05-31 22:31:39.735407  
2 2018-05-31 22:31:39.735431  
3 2018-05-31 22:31:39.735453  
4 2018-05-31 22:31:39.735475

Let's look at the data we can collect on a video level...

In [12]:
video_id = df['video_id'].tolist()
video_id[:2]

['IEDp9GQPk2I', 'zi1xZhE9HFQ']

In [13]:
yt.get_video_metadata(video_id[0], key, parser=P.parse_video_metadata)

OrderedDict([('video_id', 'IEDp9GQPk2I'),
             ('channel_title', 'Munchies'),
             ('channel_id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
             ('video_publish_date', datetime.datetime(2018, 5, 30, 15, 1, 36)),
             ('video_title', 'How-To Make Chocolate Eclair Doughnuts'),
             ('video_description',
              "What if Good Humor's chocolate eclair ice cream bar became a doughnut? Thanks to Matt Fein of Philadelphia's Federal Donuts, that's no longer a hypothetical question. Matt visits the MUNCHIES Test Kitchen to show us how to make the chocolate eclair donut, which gets dipped in a vanilla glaze and tossed in Nilla wafer and chocolate cookie crumble. \n\nSubscribe to Munchies here: http://bit.ly/Subscribe-to-MUNCHIES\n\nCheck out http://munchies.tv for more!\n\nFollow Munchies here:\nFacebook: http://facebook.com/munchies\nTwitter: http://twitter.com/munchies\nTumblr: http://munchies.tumblr.com\nInstagram: http://instagram.com/munchies\nPinterest: htt

In [14]:
yt.get_video_metadata(video_id[0], key, parser=P.default)

{'etag': '"DuHzAJ-eQIiCIp7p4ldoVcVAOeY/ZOYQb_WBsa2VTG_A7RLqqhBHsP0"',
 'id': 'IEDp9GQPk2I',
 'kind': 'youtube#video',
 'snippet': {'categoryId': '24',
  'channelId': 'UCaLfMkkHhSA_LaCta0BzyhQ',
  'channelTitle': 'Munchies',
  'description': "What if Good Humor's chocolate eclair ice cream bar became a doughnut? Thanks to Matt Fein of Philadelphia's Federal Donuts, that's no longer a hypothetical question. Matt visits the MUNCHIES Test Kitchen to show us how to make the chocolate eclair donut, which gets dipped in a vanilla glaze and tossed in Nilla wafer and chocolate cookie crumble. \n\nSubscribe to Munchies here: http://bit.ly/Subscribe-to-MUNCHIES\n\nCheck out http://munchies.tv for more!\n\nFollow Munchies here:\nFacebook: http://facebook.com/munchies\nTwitter: http://twitter.com/munchies\nTumblr: http://munchies.tumblr.com\nInstagram: http://instagram.com/munchies\nPinterest: https://www.pinterest.com/munchies\nFoursquare: https://foursquare.com/munchies\nMore videos from the VICE

The function also works for a list of up to 50 video ids

In [15]:
video_meta = yt.get_video_metadata(video_id, key)

Exception: Max length of list is 50!

To get around this, I suggest breaking the input into chunks

In [16]:
def chunks(list_, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(list_), n):
        yield list_[i:i + n]

In [17]:
video_meta = []
for chunk in chunks(video_id, n=40):
    vm_ = yt.get_video_metadata(chunk, key, P.parse_video_metadata)
    video_meta.extend(vm_)
len(video_id)

129

In [18]:
df_video_meta = pd.DataFrame(video_meta)
df_video_meta.head(2)

video_id channel_title                channel_id  video_publish_date  \
0  IEDp9GQPk2I      Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-05-30 15:01:36   
1  zi1xZhE9HFQ      Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-05-26 14:00:04   

                                         video_title  \
0             How-To Make Chocolate Eclair Doughnuts   
1  The Dinner Table Cures their Hangover with Bac...   

                                   video_description video_category  \
0  What if Good Humor's chocolate eclair ice crea...             24   
1  Our host Cara Nicoletti cooks for Ricky, Johnn...             24   

  video_view_count video_comment_count video_like_count video_dislike_count  \
0            88174                 202             1697                 168   
1           152704                 538             3072                 964   

                                    video_thumbnail  \
0  https://i.ytimg.com/vi/IEDp9GQPk2I/hqdefault.jpg   
1  https://i.ytimg.com/vi/zi1xZhE9HFQ/hqdefault.jpg   

                                          video_tags  \
0  how to|cooking|Munchies|Munchiestv|food|drinks...   
1  how to|cooking|Munchies|Munchiestv|food|drinks...   

             collection_date  
0 2018-05-31 22:31:41.937432  
1 2018-05-31 22:31:41.937472

For establishing relationships you can list featured channels and subscriptions

In [19]:
yt.get_featured_channels(channel_id)

{'UCaLfMkkHhSA_LaCta0BzyhQ': ['UCn8zNIfYAQNdrFRrr8oibKw',
  'UCWF0PiUvUi3Jma2oFgaiX2w',
  'UCfQDD-pbllOCXHYwiXxjJxA',
  'UCZaT_X_mc0BI-djXOlfhqWQ',
  'UCB6PV0cvJpzlcXRG7nz6PpQ',
  'UC0iwHRFpv2_fpojZgQhElEQ',
  'UC_NaA2HkWDT6dliWVcvnkuQ',
  'UCS6R2iiAJ1FvEYl4B3zmljw',
  'UC8C8WuWSsFjWFaTHcUQeQxA',
  'UC9ISPZsMaBi5mutsgX6LC1g',
  'UCiZCX1R1F3xYGbeXq1JscKA',
  'UCVfmHpXONv-LVACBV68tq5Q',
  'UC5e0xSqwDGlRg3sdvGQh7lg',
  'UClW2OsdCa2E_KkLZNpm_9VQ',
  'UC9XpoCBNvStSmp3gVf_jG1g',
  'UCflb1gG-X1dy1Ru5JIk5sPw',
  'UCNDUud96oGK5xQ9gyg913vw']}

You can save on time by using a list of inputs for some api calls:

In [20]:
channel_ids = ['UCaLfMkkHhSA_LaCta0BzyhQ', 'UC6MFZAOHXlKK1FI7V0XQVeA']

In [21]:
yt.get_featured_channels(channel_ids, key)

[{'UCaLfMkkHhSA_LaCta0BzyhQ': ['UCn8zNIfYAQNdrFRrr8oibKw',
   'UCWF0PiUvUi3Jma2oFgaiX2w',
   'UCfQDD-pbllOCXHYwiXxjJxA',
   'UCZaT_X_mc0BI-djXOlfhqWQ',
   'UCB6PV0cvJpzlcXRG7nz6PpQ',
   'UC0iwHRFpv2_fpojZgQhElEQ',
   'UC_NaA2HkWDT6dliWVcvnkuQ',
   'UCS6R2iiAJ1FvEYl4B3zmljw',
   'UC8C8WuWSsFjWFaTHcUQeQxA',
   'UC9ISPZsMaBi5mutsgX6LC1g',
   'UCiZCX1R1F3xYGbeXq1JscKA',
   'UCVfmHpXONv-LVACBV68tq5Q',
   'UC5e0xSqwDGlRg3sdvGQh7lg',
   'UClW2OsdCa2E_KkLZNpm_9VQ',
   'UC9XpoCBNvStSmp3gVf_jG1g',
   'UCflb1gG-X1dy1Ru5JIk5sPw',
   'UCNDUud96oGK5xQ9gyg913vw']},
 {'UC6MFZAOHXlKK1FI7V0XQVeA': ['UCSHsNH4FZXFeSQMJ56AdrBA']}]

Subscriptions can only done one channel at a time:

In [22]:
yt.get_subscriptions(channel_id, key)

[OrderedDict([('subscription_title', 'VICE Arabia'),
              ('subscription_channel_id', 'UCIZ3xweMcx1XUlcwRESbmBw'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2017, 11, 27, 23, 21, 22)),
              ('collection_date',
               datetime.datetime(2018, 5, 31, 22, 31, 44, 208663))]),
 OrderedDict([('subscription_title', 'First We Feast'),
              ('subscription_channel_id', 'UCPD_bxCRGpmmeQcbe2kpPaA'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
               datetime.datetime(2017, 8, 2, 17, 37, 4)),
              ('collection_date',
               datetime.datetime(2018, 5, 31, 22, 31, 44, 208694))]),
 OrderedDict([('subscription_title', 'Tonic'),
              ('subscription_channel_id', 'UCroeDtD1dtd1leuxUHDMTXQ'),
              ('subscription_kind', 'youtube#channel'),
              ('subscription_publish_date',
    

You can also get the comments for a given video

In [23]:
yt.get_video_comments(video_id[0])[:2]

[OrderedDict([('commenter_channel_url',
               'http://www.youtube.com/channel/UCYGU2AxOlL_sp2NiSOrRjHQ'),
              ('commenter_channel_display_name', 'noone mate'),
              ('comment_id', 'UgxknTqpvAGynNNboyV4AaABAg'),
              ('comment_like_count', 1),
              ('comment_publish_date',
               datetime.datetime(2018, 6, 1, 0, 4, 31)),
              ('text',
               '0:38 That was awkward as fuck, and you can tell that guy has never fished before lmao'),
              ('video_id', 'IEDp9GQPk2I'),
              ('commenter_rating', 'none'),
              ('comment_parent_id', None),
              ('collection_date',
               datetime.datetime(2018, 5, 31, 22, 31, 45, 35385))]),
 OrderedDict([('commenter_channel_url',
               'http://www.youtube.com/channel/UCHhGJoYGROULv0wOXCEtduQ'),
              ('commenter_channel_display_name', 'Ethannesss'),
              ('comment_id', 'UgxZbLmp1IrUD0E1Grd4AaABAg'),
              ('comment_

For more text we can get closed captions!

In [24]:
vid = 'hEDK3tC43SQ'

In [25]:
captions = yt.get_captions(vid, verbose=False)

In [26]:
captions

OrderedDict([('video_id', 'hEDK3tC43SQ'),
             ('caption',
              'Samantha: Ready to check out the Flavor Graveyard? Isaac: Yeah. There\'s a bunch of tombstones in here to our dearly de-pinted flavors. We retire flavors that just aren\'t selling. What do we got right here? Economic Crunch. "A delightful mash. This flavor we remember for the stock market crash on the 6th of November. We hardly knew you." Have you ever seen someone just fully break down here and just start sobbing? I have not, but I wouldn\'t be surprised. Yeah, I mean, it\'s a sad place. ♪♪ ♪♪ Damn! You\'re too young! ♪♪ ♪♪ ♪♪ ♪♪ I\'m Isaac Lappert. And as an ice-cream maker and businessman myself, I am very interested in the business of ice cream. We\'re starting in Sausalito, where I\'ll introduce you to my family\'s own Lappert\'s Ice Cream. We\'re on the smaller side with 10 shops in 2 states, but I\'m curious about the pros and cons of expanding. So, I\'m going to Jeni\'s Splendid Ice Creams, a pion

You can also get the recommended videos for any given video

In [27]:
recommended_vids = yt.get_recommended_videos(vid)

In [28]:
recommended_vids[:2]

[OrderedDict([('video_id', 'MeWMwwRfwFI'),
              ('channel_title', 'Munchies'),
              ('channel_id', 'UCaLfMkkHhSA_LaCta0BzyhQ'),
              ('video_publish_date',
               datetime.datetime(2015, 4, 16, 15, 12, 17)),
              ('video_title',
               'Making Cold-Stoned Sundaes with the Cannabis Creamery: BONG APPÉTIT'),
              ('video_description',
               'Watch the first episode of SMOKEABLES: How to Make a Gravity Bong - http://bit.ly/28XSWBi\n\nIn this episode of Bong Appetit, host Abdullah Saeed checks out Cannabis Creamery, a Sausalito, CA-based ice cream company that is producing sweet THC-infused treats in a range of fantastic flavors. From classic mint-chip to a grapefruit sorbet originally designed for the Grateful Dead, this ice cream is dankly delicious.\n\nOwner Isaac Lappert takes us on a visit to the family’s original business—Lappert’s Ice Cream—to hear Cannabis Creamery’s origin story straight from the horse’s mouth. 

In [29]:
pd.DataFrame(recommended_vids)

video_id  channel_title                channel_id  video_publish_date  \
0   MeWMwwRfwFI       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2015-04-16 15:12:17   
1   GajbZY0l7oY       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-01-10 23:58:53   
2   VMUzoVslOBQ       VICELAND  UCWF0PiUvUi3Jma2oFgaiX2w 2018-04-13 13:57:01   
3   JnmQd7kUDLo           VICE  UCn8zNIfYAQNdrFRrr8oibKw 2013-02-08 01:04:06   
4   vLQeb8LWyh0       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-01-17 15:38:24   
5   _bTabM6Nbx0       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-06-28 19:51:41   
6   dr4HHepGxxM       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2014-07-07 15:31:17   
7   CSDc7_mWIXo       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2016-07-21 00:58:03   
8   6WYR079F0xU       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2015-11-20 15:16:55   
9   TyvM2fb9kdM       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-02-15 17:07:27   
10  fPHjX6p-RbQ       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-04-23 23:07:56   
11  3CJQph5gLeE       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2014-08-19 13:28:13   
12  Yvfv0wmyGro       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-02-28 21:19:58   
13  tqPub3p3k5E  BuzzFeedVideo  UCpko_-a4wgz2u_DgDgd9fqA 2018-04-13 21:03:43   
14  ce0TNKeMK1M       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-09-15 16:32:02   
15  C9aiGLIry7g       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2014-10-15 21:23:26   
16  Map2OQRAmBk       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2015-01-16 05:41:27   
17  JjQWwhxz5rQ       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2015-01-21 06:45:03   
18  GKwt1yxnJoo       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2014-06-24 22:16:13   
19  n2BgkHV373A       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-01-09 17:52:24   
20  fXCrmV0-Ukg       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-05-15 22:55:10   
21  s5ySgsNafQE       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-01-30 00:07:09   
22  FBbBWifAdCg       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-12-14 22:46:31   
23  GAKp-QOa7fo       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2018-04-18 23:24:48   
24  3XklsSF53pc       Munchies  UCaLfMkkHhSA_LaCta0BzyhQ 2017-11-27 18:27:20   

                                          video_title  \
0   Making Cold-Stoned Sundaes with the Cannabis C...   
1                                The Pizza Show: Rome   
2   Corned Beef On My Mind: IT'S SUPPERTIME! (Full...   
3                            Munchies: Christina Tosi   
4   Family Food: One of a Kind Italian Deli Food a...   
5   Chef's Night Out in New York City with Scarr's...   
6   FARANG: The Story of Chef Andy Ricker of Pok P...   
7                     Bud and Breakfast: BONG APPÉTIT   
8                            BBQ Road Trip: Tennessee   
9                           The Pizza Show - Bay Area   
10                             The Pizza Show: Sicily   
11  The Best Fish 'n' Chips in the World: MUNCHIES...   
12  MUNCHIES Presents: The Art Of Making Danish Ho...   
13                                $5 Pie Vs. $250 Pie   
14  From Sea to Table: The MUNCHIES Guide to Norwa...   
15                                How to Make Ceviche   
16     Searching for East Germany's Communist Cuisine   
17  Foie Gras & the Ethics of Force-Feeding: The P...   
18    MUNCHIES Presents: Skylight Inn's Whole Hog BBQ   
19                            The Pizza Show: Detroit   
20  How-To: Make Shrimp Parm on Steroids with Meyh...   
21                     How-To Make Potato Skin Nachos   
22    The Pizza Show: Naples, The Birthplace of Pizza   
23                         Todos los Tacos: Al Pastor   
24   How-To Make a Steak Sandwich with Matty Matheson   

                                    video_description video_category  \
0   Watch the first episode of SMOKEABLES: How to ...           None   
1   When in Rome...The Pizza Show is gonna eat a l...           None   
2   A Jewish Deli in your own home! Matty shows yo...           None   
3   This ridiculous mix of fun and sweet treats is...           None   
4   In this episode of Family Food, we head to Ton...           None   
5   Born an